# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff9d9649850>
├── 'a' --> tensor([[-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902]])
└── 'x' --> <FastTreeValue 0x7ff9d9649ca0>
    └── 'c' --> tensor([[ 0.6249,  0.4188,  0.2662,  0.2933],
                        [-0.0061, -0.1024, -0.6976, -0.2532],
                        [-0.4155, -0.1005,  1.6274, -0.1806]])

In [4]:
t.a

tensor([[-1.1733,  0.4867, -1.2827],
        [ 0.0635, -0.0662, -1.2902]])

In [5]:
%timeit t.a

66 ns ± 0.848 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff9d9649850>
├── 'a' --> tensor([[-0.9687,  0.4420,  0.5520],
│                   [ 0.0688,  2.0158, -1.2026]])
└── 'x' --> <FastTreeValue 0x7ff9d9649ca0>
    └── 'c' --> tensor([[ 0.6249,  0.4188,  0.2662,  0.2933],
                        [-0.0061, -0.1024, -0.6976, -0.2532],
                        [-0.4155, -0.1005,  1.6274, -0.1806]])

In [7]:
%timeit t.a = new_value

72 ns ± 1.63 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1733,  0.4867, -1.2827],
               [ 0.0635, -0.0662, -1.2902]]),
    x: Batch(
           c: tensor([[ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806]]),
       ),
)

In [10]:
b.a

tensor([[-1.1733,  0.4867, -1.2827],
        [ 0.0635, -0.0662, -1.2902]])

In [11]:
%timeit b.a

61.6 ns ± 0.849 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2047,  0.4311, -0.0284],
               [ 1.1705, -1.1640,  0.9692]]),
    x: Batch(
           c: tensor([[ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806]]),
       ),
)

In [13]:
%timeit b.a = new_value

561 ns ± 8.42 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

944 ns ± 26.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

262 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

252 µs ± 5.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff9d95f29a0>
├── 'a' --> tensor([[[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]],
│           
│                   [[-1.1733,  0.4867, -1.2827],
│                    [ 0.0635, -0.0662, -1.2902]]])
└── 'x' --> <FastTreeValue 0x7ff91fe8b520>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.4 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff91f7d2f10>
├── 'a' --> tensor([[-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902],
│                   [-1.1733,  0.4867, -1.2827],
│                   [ 0.0635, -0.0662, -1.2902]])
└── 'x' --> <FastTreeValue 0x7ff91f7d2370>
    └── 'c' --> tensor([[ 0.6249,  0.4188,  0.2662,  0.2933],
                        [-0.0061, -0.1024, -0.6976, -0.2532],
                 

In [23]:
%timeit t_cat(trees)

41.3 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.8 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6249,  0.4188,  0.2662,  0.2933],
                       [-0.0061, -0.1024, -0.6976, -0.2532],
                       [-0.4155, -0.1005,  1.6274, -0.1806]],
              
                      [[ 0.6249,  0.4188,  0.2662,  0.2933],
                       [-0.0061, -0.1024, -0.6976, -0.2532],
                       [-0.4155, -0.1005,  1.6274, -0.1806]],
              
                      [[ 0.6249,  0.4188,  0.2662,  0.2933],
                       [-0.0061, -0.1024, -0.6976, -0.2532],
                       [-0.4155, -0.1005,  1.6274, -0.1806]],
              
                      [[ 0.6249,  0.4188,  0.2662,  0.2933],
                       [-0.0061, -0.1024, -0.6976, -0.2532],
                       [-0.4155, -0.1005,  1.6274, -0.1806]],
              
                      [[ 0.6249,  0.4188,  0.2662,  0.2933],
                       [-0.0061, -0.1024, -0.6976, -0.2532],
                       [-0.4155, -0.1005,  1.6274, -0.1806]],

In [26]:
%timeit Batch.stack(batches)

95.1 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806],
                      [ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806],
                      [ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806],
                      [ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806],
                      [ 0.6249,  0.4188,  0.2662,  0.2933],
                      [-0.0061, -0.1024, -0.6976, -0.2532],
                      [-0.4155, -0.1005,  1.6274, -0.1806],
                      [ 0.6249,  0.4188,  0.2662,  0.2933],
                   

In [28]:
%timeit Batch.cat(batches)

172 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

517 µs ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
